In [2]:
import seml
import pandas as pd
import numpy as np
from run_seml import run
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [3]:
%load_ext autoreload
%autoreload 2

In [117]:
df_experiments = seml.get_results('kdd21_local_attack_citeseer_eval',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [118]:
#df_experiments["config.dataset"].unique()

In [119]:
df_results = [
    pd.DataFrame(r) 
    for r in df_experiments['result.results']
]

In [120]:
pd.concat(df_results, ignore_index=True)["margin_diff"]

0     -0.007355
1     -0.001722
2      0.010363
3      0.089794
4      0.003082
         ...   
718   -0.004864
719    0.001399
720    0.000000
721   -0.000567
722   -0.000567
Name: margin_diff, Length: 723, dtype: float64

In [108]:
pd.concat(df_results, ignore_index=True)["margin_diff"]

0     -0.007355
1     -0.001722
2      0.010363
3      0.089794
4      0.003082
         ...   
718   -0.004864
719    0.001399
720    0.000000
721    0.000000
722    0.000000
Name: margin_diff, Length: 723, dtype: float64

In [65]:
for df_result, (_, df_experiment) in zip(df_results, df_experiments.iterrows()):
    df_result['dataset'] = df_experiment['config.dataset']
    df_result['attack'] = df_experiment['config.attack']
    df_result['seed'] = df_experiment['config.seed']
    df_result['batch_id'] = df_experiment['batch_id']

df_results = pd.concat(df_results, ignore_index=True)
df_results = df_results.sort_values('batch_id')
# df_results = df_results.drop_duplicates([
#     c for c in df_results.columns if c != 'batch_id' and c != 'accuracy'
# ], keep='last')

df_results

,label,epsilon,n_perturbations,degree,logits,initial_logits,larget,node_id,perturbed_edges,logit_target,logit_best_non_target,confidence_target,confidence_non_target,margin,initial_logit_target,initial_logit_best_non_target,initial_confidence_target,initial_confidence_non_target,initial_margin,dataset,attack,seed,batch_id
0,Vanilla GCN,0.50,2,4,"[[tensor(-4.9452), tensor(-4.8286), tensor(-0....","[[tensor(-3.2740), tensor(-2.9485), tensor(-2....",4,1132,"[[1132, 1132], [1582, 1901]]",-2.823853,-0.102808,0.059377,0.902300,-0.842924,-0.395368,-2.088969,0.673432,0.123815,0.549617,citeseer,LocalPRBCD,0,1
711,Vanilla PPRGo,0.50,2,3,"[[tensor(-1.9587), tensor(-1.8669), tensor(-2....","[[tensor(-1.8817), tensor(-0.9742), tensor(-2....",1,60,"[[60, 60], [406, 1637]]",-1.866867,-0.711566,0.154607,0.490875,-0.336267,-0.974219,-1.590702,0.377487,0.203783,0.173704,citeseer,LocalPRBCD,5,1
712,Vanilla PPRGo,0.75,2,3,"[[tensor(-1.9587), tensor(-1.8669), tensor(-2....","[[tensor(-1.8817), tensor(-0.9742), tensor(-2....",1,60,"[[60, 60], [406, 1637]]",-1.866867,-0.711566,0.154607,0.490875,-0.336267,-0.974219,-1.590702,0.377487,0.203783,0.173704,citeseer,LocalPRBCD,5,1
713,Vanilla PPRGo,1.00,3,3,"[[tensor(-2.0504), tensor(-2.0509), tensor(-2....","[[tensor(-1.8817), tensor(-0.9742), tensor(-2....",1,60,"[[60, 60, 60], [406, 924, 1637]]",-2.050868,-0.590976,0.128623,0.553787,-0.425164,-0.974219,-1.590702,0.377487,0.203783,0.173704,citeseer,LocalPRBCD,5,1
714,Vanilla PPRGo,0.50,1,2,"[[tensor(-0.5598), tensor(-1.5981), tensor(-3....","[[tensor(-1.6985), tensor(-0.5926), tensor(-3....",1,1033,"[[603], [1033]]",-1.598062,-0.559828,0.202288,0.571308,-0.369019,-0.592619,-1.698549,0.552877,0.182949,0.369928,citeseer,LocalPRBCD,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,Soft Median RPPRGo (T=1.0),0.75,3,4,"[[tensor(-2.7665), tensor(-3.1968), tensor(-0....","[[tensor(-2.7665), tensor(-3.1968), tensor(-0....",4,1132,[],-1.568240,-0.575277,0.208412,0.562549,-0.354137,-1.568240,-0.575277,0.208412,0.562549,-0.354137,citeseer,LocalPRBCD,5,2
1322,Soft Median RPPRGo (T=1.0),1.00,4,4,"[[tensor(-2.7665), tensor(-3.1968), tensor(-0....","[[tensor(-2.7665), tensor(-3.1968), tensor(-0....",4,1132,[],-1.568240,-0.575277,0.208412,0.562549,-0.354137,-1.568240,-0.575277,0.208412,0.562549,-0.354137,citeseer,LocalPRBCD,5,2
1323,Soft Median RPPRGo (T=1.0),0.50,2,4,"[[tensor(-1.9160), tensor(-0.7454), tensor(-2....","[[tensor(-1.9160), tensor(-0.7454), tensor(-2....",1,1824,[],-0.745410,-1.823979,0.474539,0.161382,0.313157,-0.745410,-1.823979,0.474539,0.161382,0.313157,citeseer,LocalPRBCD,5,2
1325,Soft Median RPPRGo (T=1.0),1.00,4,4,"[[tensor(-1.9160), tensor(-0.7454), tensor(-2....","[[tensor(-1.9160), tensor(-0.7454), tensor(-2....",1,1824,[],-0.745410,-1.823979,0.474539,0.161382,0.313157,-0.745410,-1.823979,0.474539,0.161382,0.313157,citeseer,LocalPRBCD,5,2


In [73]:
df_results["label"].unique()

array(['Vanilla GCN', 'Vanilla PPRGo', 'Soft Median RPPRGo (T=0.2)',
       'Soft Median RPPRGo (T=0.5)', 'Soft Median RPPRGo (T=1.0)'],
      dtype=object)

In [67]:
dataset_map = {
    'cora_ml': r'Cora ML', 
    'citeseer': r'Citeseer', 
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [68]:
attack_map = {
    'PRBCD': r'PR-BCD',
    'LocalPRBCD': r'Local PRBCD',
    'Nettack': r'Nettack'
}
attack_order = [attack_map[k] for k in attack_map.keys()]

In [69]:
def transform_label(label: str):
    return label

In [70]:
def calc_mean_and_error(values: pd.Series, seeds: pd.Series = None, with_error=True, decimal_places: int = 3): 
    if seeds is not None:
        values, seeds = values.values, seeds.values
        seeds = seeds[~np.isnan(values)]
        values = values[~np.isnan(values)]

        idx = np.unique(seeds, return_index=True)[1]
        values = values[idx]
    
    if with_error:
        return rf'{np.mean(values):.{decimal_places}f} $\pm$ {np.std(values)/len(values):.{decimal_places}f}'
    else:
        return rf'{np.mean(values):.{decimal_places}f}'

In [71]:
architecture_c = r'Architecture'
dataset_c = r'   '
attack_c = r'Attack'
epsilons_c = r'Frac. edges (epsilon)'

epsilons = [0.05, 0.1, 0.25]
epsilon_marks = [r'\textit', r'\underline', r'\textbf']

In [72]:

df_cora = []
df_citeseer = []

for (dataset, attack, label, epsilon), df_group in df_results.groupby(['dataset', 'attack', 'label', 'epsilon']):

       initial_margin = calc_mean_and_error(df_group.initial_margin, with_error=True)
       margin = calc_mean_and_error(df_group.margin, df_group.seed, with_error=True)
       
       confidence_target = calc_mean_and_error(df_group.confidence_target, with_error=False)
       confidence_non_target = calc_mean_and_error(df_group.confidence_non_target, with_error=False)
       initial_confidence_target = calc_mean_and_error(df_group.initial_confidence_target, with_error=False)
       initial_confidence_non_target = calc_mean_and_error(df_group.initial_confidence_non_target, with_error=False)
       if dataset == "citeseer":
              df_citeseer.append({
                     dataset_c: dataset_map[dataset],
                     architecture_c: transform_label(label),
                     attack_c: attack_map[attack],
                     epsilons_c: epsilon,
                     'initial_margin': initial_margin,
                     'margin': margin,
                     'confidence_target': confidence_target,
                     'confidence_non_target': confidence_non_target,
                     'initial_confidence_target': initial_confidence_target,
                     'initial_confidence_non_target': initial_confidence_non_target,
              })
       elif dataset == "cora_ml":
              df_cora.append({
                     dataset_c: dataset_map[dataset],
                     architecture_c: transform_label(label),
                     attack_c: attack_map[attack],
                     epsilons_c: epsilon,
                     'initial_margin': initial_margin,
                     'margin': margin,
                     'confidence_target': confidence_target,
                     'confidence_non_target': confidence_non_target,
                     'initial_confidence_target': initial_confidence_target,
                     'initial_confidence_non_target': initial_confidence_non_target,
              })


df_cora = pd.DataFrame(df_cora)
df_citeseer = pd.DataFrame(df_citeseer)

In [34]:

print(df_cora.to_markdown())

In [35]:
df_cora

""


In [36]:
df_results[df_results["label"]=="Vanilla PPRGo"].sort_values(["node_id", "n_perturbations", "seed", "epsilon", ])

,label,epsilon,n_perturbations,degree,logits,initial_logits,larget,node_id,perturbed_edges,logit_target,logit_best_non_target,confidence_target,confidence_non_target,margin,initial_logit_target,initial_logit_best_non_target,initial_confidence_target,initial_confidence_non_target,initial_margin,dataset,attack,seed,batch_id
81,Vanilla PPRGo,0.50,2,4,"[[tensor(-4.1921), tensor(-5.2794), tensor(-3....","[[tensor(-4.1921), tensor(-5.2794), tensor(-3....",4,2,[],-0.087281,-3.547653,0.916419,0.028792,0.887627,-0.087281,-3.547653,0.916419,0.028792,0.887627,citeseer,LocalPRBCD,0,1
201,Vanilla PPRGo,0.50,2,4,"[[tensor(-4.2226), tensor(-4.2111), tensor(-3....","[[tensor(-4.2226), tensor(-4.2111), tensor(-3....",4,2,[],-0.102468,-3.456214,0.902607,0.031549,0.871058,-0.102468,-3.456214,0.902607,0.031549,0.871058,citeseer,LocalPRBCD,1,1
321,Vanilla PPRGo,0.50,2,4,"[[tensor(-1.9614), tensor(-3.1962), tensor(-2....","[[tensor(-1.9614), tensor(-3.1962), tensor(-2....",4,2,[],-0.844676,-1.801137,0.429697,0.165111,0.264586,-0.844676,-1.801137,0.429697,0.165111,0.264586,citeseer,LocalPRBCD,5,1
82,Vanilla PPRGo,0.75,3,4,"[[tensor(-4.1921), tensor(-5.2794), tensor(-3....","[[tensor(-4.1921), tensor(-5.2794), tensor(-3....",4,2,[],-0.087281,-3.547653,0.916419,0.028792,0.887627,-0.087281,-3.547653,0.916419,0.028792,0.887627,citeseer,LocalPRBCD,0,1
202,Vanilla PPRGo,0.75,3,4,"[[tensor(-4.2226), tensor(-4.2111), tensor(-3....","[[tensor(-4.2226), tensor(-4.2111), tensor(-3....",4,2,[],-0.102468,-3.456213,0.902607,0.031549,0.871058,-0.102468,-3.456213,0.902607,0.031549,0.871058,citeseer,LocalPRBCD,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Vanilla PPRGo,0.75,3,4,"[[tensor(-3.4144), tensor(-3.3409), tensor(-0....","[[tensor(-3.4144), tensor(-3.3409), tensor(-0....",2,2097,[],-0.162039,-3.340863,0.850408,0.035406,0.815002,-0.162039,-3.340863,0.850408,0.035406,0.815002,citeseer,LocalPRBCD,1,1
313,Vanilla PPRGo,0.75,3,4,"[[tensor(-1.4816), tensor(-2.2674), tensor(-0....","[[tensor(-1.4816), tensor(-2.2674), tensor(-0....",2,2097,[],-0.740443,-1.481567,0.476903,0.227281,0.249621,-0.740443,-1.481567,0.476903,0.227281,0.249621,citeseer,LocalPRBCD,5,1
74,Vanilla PPRGo,1.00,4,4,"[[tensor(-4.0898), tensor(-4.4470), tensor(-0....","[[tensor(-4.0898), tensor(-4.4470), tensor(-0....",2,2097,[],-0.077421,-4.019835,0.925500,0.017956,0.907544,-0.077421,-4.019835,0.925500,0.017956,0.907544,citeseer,LocalPRBCD,0,1
194,Vanilla PPRGo,1.00,4,4,"[[tensor(-3.4144), tensor(-3.3409), tensor(-0....","[[tensor(-3.4144), tensor(-3.3409), tensor(-0....",2,2097,[],-0.162039,-3.340863,0.850408,0.035406,0.815002,-0.162039,-3.340863,0.850408,0.035406,0.815002,citeseer,LocalPRBCD,1,1


In [46]:
df_results

,label,epsilon,n_perturbations,degree,logits,initial_logits,larget,node_id,perturbed_edges,logit_target,logit_best_non_target,confidence_target,confidence_non_target,margin,initial_logit_target,initial_logit_best_non_target,initial_confidence_target,initial_confidence_non_target,initial_margin,dataset,attack,seed,batch_id
0,Vanilla PPRGo,0.50,2,4,"[[tensor(-2.2240), tensor(-2.7214), tensor(-1....","[[tensor(-2.2240), tensor(-2.7214), tensor(-1....",4,1132,[],-0.960408,-1.207005,0.382737,0.299092,0.083645,-0.960408,-1.207005,0.382737,0.299092,0.083645,citeseer,LocalPRBCD,0,1
964,Soft Median RPPRGo (T=0.5),0.75,3,4,"[[tensor(-2.0328), tensor(-1.0638), tensor(-2....","[[tensor(-2.0328), tensor(-1.0638), tensor(-2....",1,1824,[],-1.063771,-1.348494,0.345152,0.259631,0.085521,-1.063771,-1.348494,0.345152,0.259631,0.085521,citeseer,LocalPRBCD,5,1
963,Soft Median RPPRGo (T=0.5),0.50,2,4,"[[tensor(-2.0328), tensor(-1.0638), tensor(-2....","[[tensor(-2.0328), tensor(-1.0638), tensor(-2....",1,1824,[],-1.063771,-1.348494,0.345152,0.259631,0.085521,-1.063771,-1.348494,0.345152,0.259631,0.085521,citeseer,LocalPRBCD,5,1
962,Soft Median RPPRGo (T=0.5),1.00,4,4,"[[tensor(-2.6215), tensor(-2.8150), tensor(-0....","[[tensor(-2.6215), tensor(-2.8150), tensor(-0....",4,1132,[],-1.616667,-0.578748,0.198559,0.560600,-0.362040,-1.616667,-0.578748,0.198559,0.560600,-0.362040,citeseer,LocalPRBCD,5,1
961,Soft Median RPPRGo (T=0.5),0.75,3,4,"[[tensor(-2.6215), tensor(-2.8150), tensor(-0....","[[tensor(-2.6215), tensor(-2.8150), tensor(-0....",4,1132,[],-1.616667,-0.578748,0.198559,0.560600,-0.362040,-1.616667,-0.578748,0.198559,0.560600,-0.362040,citeseer,LocalPRBCD,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Soft Median RPPRGo (T=1.0),1.00,2,2,"[[tensor(-1.3277), tensor(-1.0441), tensor(-4....","[[tensor(-1.3277), tensor(-1.0441), tensor(-4....",1,1033,[],-1.044066,-1.327670,0.352021,0.265094,0.086926,-1.044066,-1.327670,0.352021,0.265094,0.086926,citeseer,LocalPRBCD,0,1
475,Soft Median RPPRGo (T=1.0),0.75,2,2,"[[tensor(-1.3277), tensor(-1.0441), tensor(-4....","[[tensor(-1.3277), tensor(-1.0441), tensor(-4....",1,1033,[],-1.044066,-1.327670,0.352021,0.265094,0.086926,-1.044066,-1.327670,0.352021,0.265094,0.086926,citeseer,LocalPRBCD,0,1
474,Soft Median RPPRGo (T=1.0),0.50,1,2,"[[tensor(-1.3277), tensor(-1.0441), tensor(-4....","[[tensor(-1.3277), tensor(-1.0441), tensor(-4....",1,1033,[],-1.044066,-1.327670,0.352021,0.265094,0.086926,-1.044066,-1.327670,0.352021,0.265094,0.086926,citeseer,LocalPRBCD,0,1
472,Soft Median RPPRGo (T=1.0),0.75,2,3,"[[tensor(-1.8802), tensor(-0.4968), tensor(-2....","[[tensor(-1.8802), tensor(-0.4968), tensor(-2....",1,60,[],-0.496810,-1.880182,0.608469,0.152562,0.455906,-0.496810,-1.880182,0.608469,0.152562,0.455906,citeseer,LocalPRBCD,0,1
